# Guía 4 - Problema 2

## Red Erdös-Rényi

In [ ]:
import random
random.seed(42)

In [2]:
# Primer caso 
N = 10000
p = 0.01

# Creamos lista de nodos
nodes = [i for i in range(N)]

# Ahora, para cada par de nodos (i,j) posible, evaluamos si hay que generar o no enlace. 
edges = []

for node_i in nodes:
    for node_j in range(node_i+1,N): # El +1 es para evitar enlaces de un nodo consigo mismo
        if random.random()<p:
            edges.append((node_i,node_j))

La red entonces está representada como una lista de enlaces, que mediante otros algoritmos puede utilizarse para construir las listas de vecinos, o la matriz de adyacencia.

El número de enlaces en la red, dividido el número máximo posible de enlaces que puede haber en un grafo de $N$ nodos, debería dar un valor cercano a $p$ (es una forma de chequear que nuestro algoritmo funciona bien).

In [3]:
import math
len(edges)/math.comb(N,2)

0.009966996699669967

Para la red más grande de $N=10^5$ vamos a implementar una pequeña paralelización que nos va a agilizar un poco el trabajo con la librería `loky`. La librería `tqdm` nos permite agregar una barra de carga para ver el tiempo restante de ejecución de manera interactiva. 

In [4]:
from loky import ProcessPoolExecutor
from tqdm import tqdm

In [5]:
# Segundo caso
N = 100000
p = 0.001

# Creamos lista de nodos
nodes = [i for i in range(N)]

# Ahora, para cada par de nodos (i,j) posible, evaluamos si hay que generar o no enlace. 
edges = []

# Esta función evalúa cuántos nodos j están enlazados con un nodo i (0<i<N, i<j<N). De esta forma no hay enlaces repetidos.
def f(node_i):
    edges_i = []
    for node_j in range(node_i+1,N): 
        if random.random() < p:
            edges_i.append((node_i,node_j))
    return edges_i

with ProcessPoolExecutor() as e:
    edges = list(tqdm(e.map(f,nodes),total=len(nodes)))

100%|██████████| 100000/100000 [02:09<00:00, 770.77it/s]


In [7]:
import numpy as np
edges = list(filter(None,edges)) # Eliminamos cualquier lista vacía que haya resultado de evaluar la función f sobre algún nodo
edges = np.concatenate(edges) # Concatenamos todos los enlaces en una única lista

In [8]:
edges

array([[    0,  1293],
       [    0,  4444],
       [    0,  5101],
       ...,
       [99922, 99944],
       [99951, 99958],
       [99994, 99995]])

In [9]:
# Chequeo
len(edges)/math.comb(N,2)

0.0010002490024900248

## Red Barabási-Albert

Por la forma en la que se construye la red en el modelo, es conveniente trabajar esta vez con las listas de vecinos para almacenar el grafo.

Vamos a empezar con un grafo aleatorio Erdös-Renyi, que cumpla con la condición pedida ($<k> = 4$). Eso se puede pedir eligiendo valores adecuados para los valores de $p$ y el número inicial de nodos en el grafo $n_0$. 

Estas variables están relacionadas mediante la siguiente ecuación: 

$$ <k> = n_0 p, $$ cuya validez es mayor a medida que $n_0\to\infty$.

Por lo tanto, vamos a elegir arbitrariamente $n_0 = N/2$, es decir que si esperamos al final un grafo de $N=10^4$ nodos, vamos a tener 5000 grafos inicialmente en el grafo aleatorio. Con esa elección, nos queda $p = \frac{2<k>}{N} = \frac{2\times 4}{10000} =0.0008$ 

In [78]:
# Primer caso
## Paso 1: Construcción del grafo aleatorio
n0 = 5000
N = 10000
p = 0.0008

# Creamos lista de nodos
nodes = [i for i in range(n0)]

# Ahora, para cada par de nodos (i,j) posible, evaluamos si hay que generar o no enlace. 
edges = []

for node_i in nodes:
    for node_j in range(node_i+1,n0): # El +1 es para evitar enlaces de un nodo consigo mismo
        if random.random()<p:
            edges.append((node_i,node_j))

In [79]:
## Paso 2: Traducción de la lista de enlaces a lista de vecinos
neighbor_list = {}

for i in range(n0):
    neighbor_list[i] = []

for i,j in edges: 
    neighbor_list[i].append(j)
    neighbor_list[j].append(i)

In [80]:
# Calculemos cuál es el grado medio del grafo aleatorio (debería ser algo cercano a 4)
k = 0
for i in range(len(neighbor_list)):
    k += len(neighbor_list[i])

print('Mean degree:',k/n0)

Mean degree: 3.9848


In [81]:
## Tercer paso: Construcción de la red de Barabási-Albert
# Lo primero que tenemos que tener es una función que nos devuelva la probabilidad de que el nodo i sea elegido en el sorteo
def get_pi(neighbor_list):
    p = []
    k_total = sum(len(neighbor_list[i]) for i in range(len(neighbor_list)))
    for i in range(len(neighbor_list)):
        p.append(len(neighbor_list[i])/k_total)
    return p

In [82]:
sum(get_pi(neighbor_list))

0.9999999999999848

In [83]:
random_state = np.random.RandomState(42)

In [84]:
# Lo segundo que tenemos que hacer es añadir nuevos nodos con m conexiones cada uno, hasta llegar a los N nodos
m = 4 
for i in range(n0,N):
    # Añadimos el nuevo nodo
    neighbor_list[i] = []
    # Sorteamos los m vecinos de este nuevo nodo con las probabilidades calculadas según la función get_pi
    new_neighbors = random_state.choice(list(neighbor_list.keys()),size=m,p=get_pi(neighbor_list))
    for new_neighbor in new_neighbors:
        neighbor_list[i].append(new_neighbor)
        neighbor_list[new_neighbor].append(i)

In [85]:
len(neighbor_list)

10000

In [93]:
# Segundo caso 
## Paso 1: Construcción del grafo aleatorio (los valores están calculados siguiendo el mismo razonamiento que en la red anterior)
n0 = 50000
N = 100000
p = 0.00008

# Creamos lista de nodos
nodes = [i for i in range(n0)]

# Ahora, para cada par de nodos (i,j) posible, evaluamos si hay que generar o no enlace. 
edges = []

# PARALELIZACIÓN: Redefinimos f para el valor n0
def f(node_i):
    edges_i = []
    for node_j in range(node_i+1,n0): 
        if random.random() < p:
            edges_i.append((node_i,node_j))
    return edges_i

with ProcessPoolExecutor() as e:
    edges = list(tqdm(e.map(f,nodes),total=len(nodes)))

edges = list(filter(None,edges)) # Eliminamos cualquier lista vacía que haya resultado de evaluar la función f sobre algún nodo
edges = np.concatenate(edges) # Concatenamos todos los enlaces en una única lista

100%|██████████| 50000/50000 [00:34<00:00, 1430.21it/s]


In [96]:
neighbor_list = {}

for i in range(n0):
    neighbor_list[i] = []

for i,j in edges: 
    neighbor_list[i].append(j)
    neighbor_list[j].append(i)

In [97]:
## Paso 2: Traducción de la lista de enlaces a lista de vecinos
# Calculemos cuál es el grado medio del grafo aleatorio (debería ser algo cercano a 4)
k = 0
for i in range(len(neighbor_list)):
    k += len(neighbor_list[i])

print('Mean degree:',k/n0)

Mean degree: 3.97664


In [98]:
## Tercer paso: Construcción de la red de Barabási-Albert (esta parte del trabajo no es paralelizable, puede demorar varios minutos la ejecución)
m = 4
for i in range(n0,N):
    # Añadimos el nuevo nodo
    neighbor_list[i] = []
    # Sorteamos los m vecinos de este nuevo nodo con las probabilidades calculadas según la función get_pi
    new_neighbors = random_state.choice(list(neighbor_list.keys()),size=m,p=get_pi(neighbor_list))
    for new_neighbor in new_neighbors:
        neighbor_list[i].append(new_neighbor)
        neighbor_list[new_neighbor].append(i)

In [99]:
len(neighbor_list)

100000